In [1]:
import colorsys,sys,re
import os
import ipywidgets
from biopandas.pdb import PandasPdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import Bio
from Bio.PDB import PDBParser
from scipy.spatial.distance import euclidean
import utility


In [2]:
data_dir = '../../data'
result_dir = '../../results'
ex_dir = os.path.join(data_dir, '1ma9_example')

In [3]:
ollivier_dir = os.path.join(os.getcwd(), 'ollivier')
forman_dir = os.path.join(os.getcwd(), 'forman')
gnm_dir = os.path.join(os.getcwd(), 'gnm')
geometry_dir = os.path.join(os.getcwd(), 'geometry')
fractal_dir = os.path.join(os.getcwd(), 'fractal')
osipov_dir = os.path.join(os.getcwd(), 'osipov')
hp_dir =  os.path.join(os.getcwd(), 'hp_idx')
c_charge_dir =  os.path.join(os.getcwd(), 'c_charge')
c_count_dir =  os.path.join(os.getcwd(), 'c_count')

new_pdb_dir = os.path.join(os.getcwd(), '1ma9')


In [4]:
coordA = PandasPdb()
coordA.read_pdb(os.path.join(ex_dir, '1ma9_A.pdb'))

coordB = PandasPdb()
coordB.read_pdb(os.path.join(ex_dir, '1ma9_B.pdb'))

In [5]:
def plot_dmap(pdb_id):
    coord = PandasPdb()
    coord.read_pdb(os.path.join(ex_dir, pdb_id+'.pdb'))


    xyz_CA = pd.concat([coord.df['ATOM'][coord.df['ATOM']['atom_name']=='CA']['x_coord'],
                          coord.df['ATOM'][coord.df['ATOM']['atom_name']=='CA']['y_coord'],
                          coord.df['ATOM'][coord.df['ATOM']['atom_name']=='CA']['z_coord']],axis=1).to_numpy()
    updated_dist = utility.new_dist(xyz_CA, xyz_CA)        
    
    plt.imshow(updated_dist, cmap=plt.vi
    plt.colorbar()
    

In [14]:
plot_dmap('1ma9')
plt.savefig(os.path.join(result_dir, '1ma9_distance_map.png'))

In [15]:
def CA_coord(pdb_name, chain1, chain2):
    coord = PandasPdb()
    coord.read_pdb(os.path.join(ex_dir, pdb_name+'.pdb'))
    c1 = coord.df['ATOM'][coord.df['ATOM']['chain_id']==chain1]
    c2 = coord.df['ATOM'][coord.df['ATOM']['chain_id']==chain2]
    
    c1_all = pd.concat([c1['x_coord'], c1['y_coord'], c1['z_coord']], axis=1).to_numpy()
    c2_all = pd.concat([c2['x_coord'], c2['y_coord'], c2['z_coord']], axis=1).to_numpy()
    
    c1_CA = pd.concat([c1[c1['atom_name']=='CA']['x_coord'], 
                      c1[c1['atom_name']=='CA']['y_coord'], 
                      c1[c1['atom_name']=='CA']['z_coord']], 
                      axis=1).to_numpy()

    c2_CA = pd.concat([c2[c2['atom_name']=='CA']['x_coord'], 
                      c2[c2['atom_name']=='CA']['y_coord'], 
                      c2[c2['atom_name']=='CA']['z_coord']], 
                      axis=1).to_numpy()
    
    return c1, c2, c1_CA, c2_CA

In [16]:
c1, c2, c1_CA, c2_CA = CA_coord('1ma9', 'A', 'B')
updated_dist = utility.new_dist(c1_CA, c2_CA) 

In [17]:
A_inter = np.unique(np.where(updated_dist<7)[0])
B_inter = np.unique(np.where(updated_dist<7)[1])

In [18]:
o_A = np.load(os.path.join(ex_dir, 'ollivier_1ma9_A.npy'))
f_A = np.load(os.path.join(ex_dir, 'forman_1ma9_A.npy'))
g_A = np.load(os.path.join(ex_dir, 'gnm_1ma9_A.npy'))
fractal_A = np.load(os.path.join(ex_dir, 'fd_1ma9_A.npy'))
osipov_A_7 = np.load(os.path.join(ex_dir, 'opd_1ma9_A_7.npy'))
hp_A = np.load(os.path.join(ex_dir, 'hp_1ma9_A.npy'))
c_charge_A = np.load(os.path.join(ex_dir, 'c_charge_1ma9_A.npy'))
c_count_A = np.load(os.path.join(ex_dir, 'c_count_1ma9_A.npy'))

geo_txtA = os.path.join(ex_dir, 'ge_1ma9_A.txt')
res_numA, shellAccA, RinaccA, PocketnessA = np.loadtxt(geo_txtA, skiprows=43, usecols=(0,3,4,7), unpack=True)

resA = coordA.df['ATOM']['residue_name'][coordA.df['ATOM']['atom_name']=='CA']

mw_A = np.zeros(len(resA))
for ii in range(len(resA)):
    if resA.iloc[ii] == 'ALA':
        mw_A[ii] = 89.1
    if resA.iloc[ii] == 'ARG':
        mw_A[ii] = 174.2
    if resA.iloc[ii] == 'ASN':
        mw_A[ii] = 132.1
    if resA.iloc[ii] == 'ASP':
        mw_A[ii] = 133.1
    if resA.iloc[ii] == 'CYS':
        mw_A[ii] = 121.2
    if resA.iloc[ii] == 'GLN':
        mw_A[ii] = 146.2
    if resA.iloc[ii] == 'GLU':
        mw_A[ii] = 147.1
    if resA.iloc[ii] == 'GLY':
        mw_A[ii] = 75.1
    if resA.iloc[ii] == 'HIS':
        mw_A[ii] = 155.2
    if resA.iloc[ii] == 'ILE':
        mw_A[ii] = 131.2
    if resA.iloc[ii] == 'LEU':
        mw_A[ii] = 131.2
    if resA.iloc[ii] == 'LYS':
        mw_A[ii] = 146.2
    if resA.iloc[ii] == 'MET':
        mw_A[ii] = 149.2
    if resA.iloc[ii] == 'PHE':
        mw_A[ii] = 165.2
    if resA.iloc[ii] == 'PRO':
        mw_A[ii] = 115.1
    if resA.iloc[ii] == 'SER':
        mw_A[ii] = 105.1
    if resA.iloc[ii] == 'THR':
        mw_A[ii] = 119.1
    if resA.iloc[ii] == 'TRP':
        mw_A[ii] = 204.2
    if resA.iloc[ii] == 'TYR':
        mw_A[ii] = 181.2
    if resA.iloc[ii] == 'VAL':
        mw_A[ii] = 117.1

In [19]:
o_B = np.load(os.path.join(ex_dir, 'ollivier_1ma9_B.npy'))
f_B = np.load(os.path.join(ex_dir, 'forman_1ma9_B.npy'))
g_B = np.load(os.path.join(ex_dir, 'gnm_1ma9_B.npy'))
fractal_B = np.load(os.path.join(ex_dir, 'fd_1ma9_B.npy'))
osipov_B_7 = np.load(os.path.join(ex_dir, 'opd_1ma9_B_7.npy'))
hp_B = np.load(os.path.join(ex_dir, 'hp_1ma9_B.npy'))
c_charge_B = np.load(os.path.join(ex_dir, 'c_charge_1ma9_B.npy'))
c_count_B = np.load(os.path.join(ex_dir, 'c_count_1ma9_B.npy'))

geo_txtB = os.path.join(ex_dir, 'ge_1ma9_B.txt')
res_numB, shellAccB, RinaccB, PocketnessB = np.loadtxt(geo_txtB, skiprows=43, usecols=(0,3,4,7), unpack=True)

resB = coordB.df['ATOM']['residue_name'][coordB.df['ATOM']['atom_name']=='CA']

mw_B = np.zeros(len(resB))
for ii in range(len(resB)):
    if resB.iloc[ii] == 'ALA':
        mw_B[ii] = 89.1
    if resB.iloc[ii] == 'ARG':
        mw_B[ii] = 174.2
    if resB.iloc[ii] == 'ASN':
        mw_B[ii] = 132.1
    if resB.iloc[ii] == 'ASP':
        mw_B[ii] = 133.1
    if resB.iloc[ii] == 'CYS':
        mw_B[ii] = 121.2
    if resB.iloc[ii] == 'GLN':
        mw_B[ii] = 146.2
    if resB.iloc[ii] == 'GLU':
        mw_B[ii] = 147.1
    if resB.iloc[ii] == 'GLY':
        mw_B[ii] = 75.1
    if resB.iloc[ii] == 'HIS':
        mw_B[ii] = 155.2
    if resB.iloc[ii] == 'ILE':
        mw_B[ii] = 131.2
    if resB.iloc[ii] == 'LEU':
        mw_B[ii] = 131.2
    if resB.iloc[ii] == 'LYS':
        mw_B[ii] = 146.2
    if resB.iloc[ii] == 'MET':
        mw_B[ii] = 149.2
    if resB.iloc[ii] == 'PHE':
        mw_B[ii] = 165.2
    if resB.iloc[ii] == 'PRO':
        mw_B[ii] = 115.1
    if resB.iloc[ii] == 'SER':
        mw_B[ii] = 105.1
    if resB.iloc[ii] == 'THR':
        mw_B[ii] = 119.1
    if resB.iloc[ii] == 'TRP':
        mw_B[ii] = 204.2
    if resB.iloc[ii] == 'TYR':
        mw_B[ii] = 181.2
    if resB.iloc[ii] == 'VAL':
        mw_B[ii] = 117.1

In [20]:
data_interA = {'MW': mw_A[A_inter],
    'Hp':hp_A[A_inter],
       'C_charge':c_charge_A[A_inter],
       'C_count':c_count_A[A_inter],
        'Shell': shellAccA[A_inter],
        'R$_{inacc}$': RinaccA[A_inter], 
        'Pocket': PocketnessA[A_inter],
        'OPD':osipov_A_7[A_inter],
       'ORC':o_A[A_inter],
       'FRC':f_A[A_inter], 
       'GNM':g_A[A_inter],
       'MFD':fractal_A[A_inter]
       }

df_interA = pd.DataFrame(data_interA, columns=['MW','Hp', 'C_charge', 'C_count','Shell', 'R$_{inacc}$', 'Pocket', 
                                             'OPD', 'ORC', 'FRC', 'GNM', 'MFD'])

corrMat_interA = df_interA.corr()
print(corrMat_interA)

                   MW        Hp  C_charge   C_count     Shell  R$_{inacc}$  \
MW           1.000000 -0.108532 -0.014267  0.785662 -0.561973    -0.485235   
Hp          -0.108532  1.000000 -0.838272  0.426804  0.163770     0.284606   
C_charge    -0.014267 -0.838272  1.000000 -0.596600 -0.060466    -0.203614   
C_count      0.785662  0.426804 -0.596600  1.000000 -0.466106    -0.332696   
Shell       -0.561973  0.163770 -0.060466 -0.466106  1.000000     0.919500   
R$_{inacc}$ -0.485235  0.284606 -0.203614 -0.332696  0.919500     1.000000   
Pocket       0.003794 -0.327171  0.340964 -0.117241 -0.211347    -0.558219   
OPD          0.278714 -0.336460  0.110291  0.063393 -0.301197    -0.204016   
ORC          0.059694  0.174574 -0.160142  0.085651 -0.038357     0.218373   
FRC          0.007402  0.171383 -0.156422  0.018607  0.102772     0.321250   
GNM          0.009526  0.306792 -0.299188  0.080366  0.096489     0.310008   
MFD         -0.076332  0.155620 -0.117125  0.077141  0.297528   

In [21]:
data_interB = {'MW': mw_B[B_inter],
    'Hp':hp_B[B_inter],
       'C_charge':c_charge_B[B_inter],
       'C_count':c_count_B[B_inter],
        'Shell': shellAccB[B_inter],
        'R$_{inacc}$': RinaccB[B_inter], 
        'Pocket': PocketnessA[B_inter],
        'OPD':osipov_B_7[B_inter],
       'ORC':o_B[B_inter],
       'FRC':f_B[B_inter], 
       'GNM':g_B[B_inter],
       'MFD':fractal_B[B_inter]
       }

df_interB = pd.DataFrame(data_interB, columns=['MW','Hp', 'C_charge', 'C_count','Shell', 'R$_{inacc}$', 'Pocket', 
                                             'OPD', 'ORC', 'FRC', 'GNM', 'MFD'])

corrMat_interB = df_interB.corr()
print(corrMat_interB)

                   MW        Hp  C_charge   C_count     Shell  R$_{inacc}$  \
MW           1.000000 -0.254782  0.034625  0.831511 -0.020327    -0.175095   
Hp          -0.254782  1.000000 -0.886369  0.174798 -0.066172    -0.023321   
C_charge     0.034625 -0.886369  1.000000 -0.442611  0.122081     0.130541   
C_count      0.831511  0.174798 -0.442611  1.000000 -0.049696    -0.247165   
Shell       -0.020327 -0.066172  0.122081 -0.049696  1.000000     0.918120   
R$_{inacc}$ -0.175095 -0.023321  0.130541 -0.247165  0.918120     1.000000   
Pocket      -0.254848  0.361535 -0.208310 -0.071240  0.020127    -0.098836   
OPD         -0.030305 -0.185605  0.050716 -0.133978 -0.488090    -0.435818   
ORC         -0.168697  0.171357 -0.040504 -0.159744  0.411175     0.517698   
FRC         -0.170783 -0.028453  0.157647 -0.247885  0.606337     0.674092   
GNM          0.088451  0.069642  0.024541  0.086733  0.550962     0.553182   
MFD         -0.089636 -0.099766  0.287519 -0.197179  0.755902   

In [22]:
dataA = {'MW': mw_A,
    'Hp':hp_A,
       'C_charge':c_charge_A,
       'C_count':c_count_A,
        'Shell': shellAccA,
        'R$_{inacc}$': RinaccA, 
        'Pocket': PocketnessA,
        'OPD':osipov_A_7,
       'ORC':o_A,
       'FRC':f_A, 
       'GNM':g_A,
       'MFD':fractal_A
       }

dfA = pd.DataFrame(dataA, columns=['MW','Hp', 'C_charge', 'C_count','Shell', 'R$_{inacc}$', 'Pocket', 
                                             'OPD', 'ORC', 'FRC', 'GNM', 'MFD'])

corrMatA = dfA.corr()
print(corrMatA)

                   MW        Hp  C_charge   C_count     Shell  R$_{inacc}$  \
MW           1.000000 -0.316149 -0.020991  0.829434  0.029588    -0.015037   
Hp          -0.316149  1.000000 -0.763894  0.045826 -0.414896    -0.395461   
C_charge    -0.020991 -0.763894  1.000000 -0.489730  0.306392     0.313215   
C_count      0.829434  0.045826 -0.489730  1.000000 -0.094276    -0.139760   
Shell        0.029588 -0.414896  0.306392 -0.094276  1.000000     0.941949   
R$_{inacc}$ -0.015037 -0.395461  0.313215 -0.139760  0.941949     1.000000   
Pocket       0.086964 -0.009490 -0.042030  0.111003  0.043442    -0.253525   
OPD          0.206420 -0.019091 -0.084338  0.207151 -0.211610    -0.186272   
ORC          0.024343 -0.200949  0.152809 -0.045648  0.380866     0.451992   
FRC          0.020220 -0.237560  0.166401 -0.056830  0.542557     0.554668   
GNM          0.009905 -0.033165  0.056467 -0.046613  0.456888     0.469631   
MFD         -0.025958 -0.129577  0.116965 -0.070838  0.558900   

In [23]:
dataB = {'MW': mw_B,
    'Hp':hp_B,
       'C_charge':c_charge_B,
       'C_count':c_count_B,
        'Shell': shellAccB,
        'R$_{inacc}$': RinaccB, 
        'Pocket': PocketnessB,
        'OPD':osipov_B_7,
       'ORC':o_B,
       'FRC':f_B, 
       'GNM':g_B,
       'MFD':fractal_B
       }

dfB = pd.DataFrame(dataB, columns=['MW','Hp', 'C_charge', 'C_count','Shell', 'R$_{inacc}$', 'Pocket', 
                                             'OPD', 'ORC', 'FRC', 'GNM', 'MFD'])

corrMatB = dfB.corr()
print(corrMatB)

                   MW        Hp  C_charge   C_count     Shell  R$_{inacc}$  \
MW           1.000000 -0.284843  0.010903  0.861833  0.098928     0.071018   
Hp          -0.284843  1.000000 -0.842565  0.079445 -0.402382    -0.382485   
C_charge     0.010903 -0.842565  1.000000 -0.415666  0.334592     0.330184   
C_count      0.861833  0.079445 -0.415666  1.000000 -0.054902    -0.078991   
Shell        0.098928 -0.402382  0.334592 -0.054902  1.000000     0.955511   
R$_{inacc}$  0.071018 -0.382485  0.330184 -0.078991  0.955511     1.000000   
Pocket      -0.032726 -0.098013  0.071594 -0.047118  0.055055    -0.177539   
OPD          0.233748 -0.123312  0.069435  0.142655  0.004614     0.015150   
ORC          0.081379 -0.174548  0.123822  0.001686  0.505100     0.558410   
FRC          0.145277 -0.213320  0.142679  0.050184  0.617653     0.635937   
GNM          0.015993 -0.080593  0.083335 -0.022898  0.489089     0.563867   
MFD         -0.044807 -0.156986  0.145983 -0.109055  0.588064   

In [24]:
df = pd.concat([dfA, dfB])
df_inter = pd.concat([df_interA, df_interB])

In [25]:
#correlation_LL 
n_colors = 100 # Use 256 colors for the diverging color palette

palette = sns.diverging_palette(10, 220, n=n_colors)
# palette=sns.diverging_palette(20, 220, n=256)

color_min, color_max = [-1, 1] # Range of values that will be mapped to the palette, i.e. min and max possible correlation

def value_to_color(val):
    val_position = float((val - color_min)) / (color_max - color_min) # position of value in the input range, relative to the length of the input range
    ind = int(val_position * (n_colors - 1)) # target index in the color palette
    return palette[ind]

def heatmap(x, y, size, color):
    fig, ax = plt.subplots(figsize=(50,50))
#     plt.figure(figsize=(100,100))
    
    # Mapping from column names to integer coordinates
    x_labels = df.columns#[v for v in (x.unique())]
    y_labels = df.columns[::-1]#[v for v in (y.unique())]
    x_to_num = {p[1]:p[0] for p in enumerate(x_labels)} 
    y_to_num = {p[1]:p[0] for p in enumerate(y_labels)} 
    
    plot_grid = plt.GridSpec(1, 15, hspace=0.2, wspace=0.1) # Setup a 1x15 grid
    ax = plt.subplot(plot_grid[:,:-1]) # Use the leftmost 14 columns of the grid for the main plot
    
    size_scale = 30000
    ax.scatter(
        x=x.map(x_to_num), # Use mapping for x
        y=y.map(y_to_num), # Use mapping for y
        s=size*size_scale, # Vector of square sizes, proportional to size parameter
        c=color.apply(value_to_color),
        marker='s', edgecolor='k', linewidth=2 # Use square as scatterplot marker
    )
    
    # Show column labels on the axes
    ax.set_xticks([x_to_num[v] for v in x_labels])
    ax.set_xticklabels(x_labels, fontsize=150, rotation=45, horizontalalignment='right')
    ax.set_yticks([y_to_num[v] for v in y_labels])
    ax.set_yticklabels(y_labels, fontsize=150)

    ax.grid(False, 'major')
    ax.grid(True, 'minor', linewidth=5)
    ax.set_xticks([t + 0.5 for t in ax.get_xticks()], minor=True)
    ax.set_yticks([t + 0.5 for t in ax.get_yticks()], minor=True)
    
    ax.set_xlim([-0.5, max([v for v in x_to_num.values()]) + 0.5]) 
    ax.set_ylim([-0.5, max([v for v in y_to_num.values()]) + 0.5])
    

    # Add color legend on the right side of the plot
    ax = plt.subplot(plot_grid[:,-1]) # Use the rightmost column of the plot

    col_x = [0]*len(palette) # Fixed x coordinate for the bars
    bar_y=np.linspace(color_min, color_max, n_colors) # y coordinates for each of the n_colors bars

    bar_height = bar_y[1] - bar_y[0]
    ax.barh(
        y=bar_y,
        width=[5]*len(palette), # Make bars 5 units wide
        left=col_x, # Make bars start at 0
        height=bar_height,
        color=palette,
        linewidth=0
    )
    ax.set_xlim(1, 2) # Bars are going from 0 to 5, so lets crop the plot somewhere in the middle
    ax.grid(False) # Hide grid
#     ax.grid(True)
    ax.set_facecolor('white') # Make background white
    ax.set_xticks([]) # Remove horizontal ticks
    ax.set_yticks(np.linspace(min(bar_y), max(bar_y), 3)) # Show vertical ticks for min, middle and max
    ax.yaxis.tick_right() # Show vertical ticks on the right 
    ax.tick_params(axis="y", labelsize=70)



In [26]:
palette = sns.diverging_palette(260, 10, n=n_colors)
# sns.palplot(palette)

In [27]:
Aa = ['MW','Hydrophobicity', 'C_charge', 'C_count','Shell', 'R$_{inacc}$', 'Pocket', 
                                             'OPD', 'ORC', 'FRC', 'GNM', 'MFD']
Acorr = dfA.corr()
# corr = df_inter[Aa].corr()
Acorr = pd.melt(Acorr.reset_index(), id_vars='index') # Unpivot the dataframe, so we can get pair of arrays for x and y
Acorr.columns = ['x', 'y', 'value']

iAcorr = df_interA.corr()
# corr = df_inter[Aa].corr()
iAcorr = pd.melt(iAcorr.reset_index(), id_vars='index') # Unpivot the dataframe, so we can get pair of arrays for x and y
iAcorr.columns = ['x', 'y', 'value']


Bcorr = dfB.corr()
# corr = df_inter[Aa].corr()
Bcorr = pd.melt(Bcorr.reset_index(), id_vars='index') # Unpivot the dataframe, so we can get pair of arrays for x and y
Bcorr.columns = ['x', 'y', 'value']

iBcorr = df_interB.corr()
# corr = df_inter[Aa].corr()
iBcorr = pd.melt(iBcorr.reset_index(), id_vars='index') # Unpivot the dataframe, so we can get pair of arrays for x and y
iBcorr.columns = ['x', 'y', 'value']


corr = df.corr()
corr = pd.melt(corr.reset_index(), id_vars='index') # Unpivot the dataframe, so we can get pair of arrays for x and y
corr.columns = ['x', 'y', 'value']

icorr = df_inter.corr()
icorr = pd.melt(icorr.reset_index(), id_vars='index') # Unpivot the dataframe, so we can get pair of arrays for x and y
icorr.columns = ['x', 'y', 'value']


In [28]:
mol = corr[corr['x']=='MW']
hydro = corr[corr['x']=='Hp'].iloc[1:]
c_ch = corr[corr['x']=='C_charge'].iloc[2:]
c_co = corr[corr['x']=='C_count'].iloc[3:]
she = corr[corr['x']=='Shell'].iloc[4:]
Ri = corr[corr['x']=='R$_{inacc}$'].iloc[5:]
Poc = corr[corr['x']=='Pocket'].iloc[6:]
Osi = corr[corr['x']=='OPD'].iloc[7:]
olli = corr[corr['x']=='ORC'].iloc[8:]
form = corr[corr['x']=='FRC'].iloc[9:]
Gau = corr[corr['x']=='GNM'].iloc[10:]
Mfra = corr[corr['x']=='MFD'].iloc[11:]

corr_half = pd.concat([mol, hydro, c_ch, c_co, she, Ri, Poc, Osi, olli, form, Gau, Mfra])

In [29]:
imol = icorr[icorr['y']=='MW']
ihydro = icorr[icorr['y']=='Hp'].iloc[1:]
ic_ch = icorr[icorr['y']=='C_charge'].iloc[2:]
ic_co = icorr[icorr['y']=='C_count'].iloc[3:]
ishe = icorr[icorr['y']=='Shell'].iloc[4:]
iRi = icorr[icorr['y']=='R$_{inacc}$'].iloc[5:]
iPoc = icorr[icorr['y']=='Pocket'].iloc[6:]
iOsi = icorr[icorr['y']=='OPD'].iloc[7:]
iolli = icorr[icorr['y']=='ORC'].iloc[8:]
iform = icorr[icorr['y']=='FRC'].iloc[9:]
iGau = icorr[icorr['y']=='GNM'].iloc[10:]
iMfra = icorr[icorr['y']=='MFD'].iloc[11:]

icorr_half = pd.concat([imol, ihydro, ic_ch, ic_co, ishe, iRi, iPoc, iOsi, iolli, iform, iGau, iMfra])

In [30]:
Amol = Acorr[Acorr['x']=='MW']
Ahydro = Acorr[Acorr['x']=='Hp'].iloc[1:]
Ac_ch = Acorr[Acorr['x']=='C_charge'].iloc[2:]
Ac_co = Acorr[Acorr['x']=='C_count'].iloc[3:]
Ashe = Acorr[Acorr['x']=='Shell'].iloc[4:]
ARi = Acorr[Acorr['x']=='R$_{inacc}$'].iloc[5:]
APoc = Acorr[Acorr['x']=='Pocket'].iloc[6:]
AOsi = Acorr[Acorr['x']=='OPD'].iloc[7:]
Aolli = Acorr[Acorr['x']=='ORC'].iloc[8:]
Aform = Acorr[Acorr['x']=='FRC'].iloc[9:]
AGau = Acorr[Acorr['x']=='GNM'].iloc[10:]
AMfra = Acorr[Acorr['x']=='MFD'].iloc[11:]

Acorr_half = pd.concat([Amol, Ahydro, Ac_ch, Ac_co, Ashe, ARi, APoc, AOsi, Aolli, Aform, AGau, AMfra])

In [31]:
iAmol = iAcorr[iAcorr['y']=='MW']
iAhydro = iAcorr[iAcorr['y']=='Hp'].iloc[1:]
iAc_ch = iAcorr[iAcorr['y']=='C_charge'].iloc[2:]
iAc_co = iAcorr[iAcorr['y']=='C_count'].iloc[3:]
iAshe = iAcorr[iAcorr['y']=='Shell'].iloc[4:]
iARi = iAcorr[iAcorr['y']=='R$_{inacc}$'].iloc[5:]
iAPoc = iAcorr[iAcorr['y']=='Pocket'].iloc[6:]
iAOsi = iAcorr[iAcorr['y']=='OPD'].iloc[7:]
iAolli = iAcorr[iAcorr['y']=='ORC'].iloc[8:]
iAform = iAcorr[iAcorr['y']=='FRC'].iloc[9:]
iAGau = iAcorr[iAcorr['y']=='GNM'].iloc[10:]
iAMfra = iAcorr[iAcorr['y']=='MFD'].iloc[11:]

iAcorr_half = pd.concat([iAmol, iAhydro, iAc_ch, iAc_co, iAshe, iARi, iAPoc, iAOsi, iAolli, iAform, iAGau, iAMfra])

In [32]:
Bmol = Bcorr[Bcorr['x']=='MW']
Bhydro = Bcorr[Bcorr['x']=='Hp'].iloc[1:]
Bc_ch = Bcorr[Bcorr['x']=='C_charge'].iloc[2:]
Bc_co = Bcorr[Bcorr['x']=='C_count'].iloc[3:]
Bshe = Bcorr[Bcorr['x']=='Shell'].iloc[4:]
BRi = Bcorr[Bcorr['x']=='R$_{inacc}$'].iloc[5:]
BPoc = Bcorr[Bcorr['x']=='Pocket'].iloc[6:]
BOsi = Bcorr[Bcorr['x']=='OPD'].iloc[7:]
Bolli = Bcorr[Bcorr['x']=='ORC'].iloc[8:]
Bform = Bcorr[Bcorr['x']=='FRC'].iloc[9:]
BGau = Bcorr[Bcorr['x']=='GNM'].iloc[10:]
BMfra = Bcorr[Bcorr['x']=='MFD'].iloc[11:]

Bcorr_half = pd.concat([Bmol, Bhydro, Bc_ch, Bc_co, Bshe, BRi, BPoc, BOsi, Bolli, Bform, BGau, BMfra])

In [33]:
iBmol = iBcorr[iBcorr['y']=='MW']
iBhydro = iBcorr[iBcorr['y']=='Hp'].iloc[1:]
iBc_ch = iBcorr[iBcorr['y']=='C_charge'].iloc[2:]
iBc_co = iBcorr[iBcorr['y']=='C_count'].iloc[3:]
iBshe = iBcorr[iBcorr['y']=='Shell'].iloc[4:]
iBRi = iBcorr[iBcorr['y']=='R$_{inacc}$'].iloc[5:]
iBPoc = iBcorr[iBcorr['y']=='Pocket'].iloc[6:]
iBOsi = iBcorr[iBcorr['y']=='OPD'].iloc[7:]
iBolli = iBcorr[iBcorr['y']=='ORC'].iloc[8:]
iBform = iBcorr[iBcorr['y']=='FRC'].iloc[9:]
iBGau = iBcorr[iBcorr['y']=='GNM'].iloc[10:]
iBMfra = iBcorr[iBcorr['y']=='MFD'].iloc[11:]

iBcorr_half = pd.concat([iBmol, iBhydro, iBc_ch, iBc_co, iBshe, iBRi, iBPoc, iBOsi, iBolli, iBform, iBGau, iBMfra])

In [34]:
corr = pd.concat([Acorr_half, Bcorr_half])
icorr= pd.concat([iAcorr_half, iBcorr_half])

In [35]:
df = pd.concat([dfA, dfB])
df_inter = pd.concat([df_interA, df_interB])


In [36]:
df_corr = df.corr()
dfcorr = pd.melt(df_corr.reset_index(), id_vars='index') # Unpivot the dataframe, so we can get pair of arrays for x and y
dfcorr.columns = ['x', 'y', 'value']

idf_corr = df_inter.corr()
idfcorr = pd.melt(idf_corr.reset_index(), id_vars='index') # Unpivot the dataframe, so we can get pair of arrays for x and y
idfcorr.columns = ['x', 'y', 'value']

In [41]:
heatmap(
    x=dfcorr['x'],
    y=dfcorr['y'],
    size=(dfcorr['value'].abs()),
    color=(dfcorr['value'])
)
plt.savefig(os.path.join(result_dir, '1ma9_all_area_corr.png'))

In [42]:
heatmap(
    x=idfcorr['x'],
    y=idfcorr['y'],
    size=(idfcorr['value'].abs()),
    color=(idfcorr['value'])
)
plt.savefig(os.path.join(result_dir, '1ma9_inter_area_corr.png'))